In [ ]:
%pip install pandas sqlalchemy langchain langchain_community langchain_openai python_dotenv  --default-timeout=100

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
  Using cached openai-1.97.1-py3-none-any.whl.metadata (29 kB)
  Using cached tiktoken-0.9.0-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.10.0-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl.metadata (41 kB)
Using cached langchain_openai-0.3.28-py3-none-any.whl (70 kB)
Using cached openai-1.97.1-py3-none-any.whl (764 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.10.0-cp313-cp313-win_amd64.whl (205 kB)
   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.7 kB ? eta -:--:--
   -----

In [2]:
import pandas as pd

df = pd.read_csv("./F1.csv")

C:\Users\salva\AppData\Local\Temp\ipykernel_21104\1652706328.py:3: DtypeWarning: Columns (6,28,41,46,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./F1.csv")


# Creación de base de datos SQL a partir de dataframe (csv)

In [3]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///f1.db', echo=True)
df.to_sql('f1', con=engine, if_exists='replace', index=False)

2025-07-26 11:36:23,778 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:36:23,893 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("f1")
2025-07-26 11:36:23,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:36:23,904 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("f1")
2025-07-26 11:36:23,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:36:23,936 INFO sqlalchemy.engine.Engine 
CREATE TABLE f1 (
	"raceId" BIGINT, 
	year BIGINT, 
	round BIGINT, 
	"circuitId" BIGINT, 
	name_x TEXT, 
	date TEXT, 
	time_x TEXT, 
	"circuitRef" TEXT, 
	name_y TEXT, 
	location TEXT, 
	country TEXT, 
	lat FLOAT, 
	lng FLOAT, 
	alt BIGINT, 
	"resultId" BIGINT, 
	"driverId" BIGINT, 
	"constructorId" BIGINT, 
	number_x FLOAT, 
	grid BIGINT, 
	position FLOAT, 
	"positionText" TEXT, 
	"positionOrder" BIGINT, 
	points FLOAT, 
	laps BIGINT, 
	time_y TEXT, 
	milliseconds FLOAT, 
	"fastestLap" FLOAT, 
	rank FLOAT, 
	"fastestLapTime" TEXT, 
	"fastestLapSpeed" FLOAT, 
	"status

25121

# Creación de agente
- La temperatura es 0 para que se limite a responder con la información de la base de datos y que no se ponga creativo

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#from langchain_openai import ChatOpenAI #OpenAI LLM
from langchain_google_genai import ChatGoogleGenerativeAI #Google LLM
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

db = SQLDatabase(engine=engine)

# llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY")) #openai llm
#agente = create_sql_agent(llm=llm, database=db,agent_type="openai-tools" , verbose=True) #verbose=True para ver como "piensa" el agente

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, google_api_key=os.getenv("GOOGLE_API_KEY")) #google llm
agente = create_sql_agent(llm=llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 

2025-07-26 11:43:29,736 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:43:29,738 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-07-26 11:43:29,739 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:43:29,741 INFO sqlalchemy.engine.Engine ROLLBACK
2025-07-26 11:43:29,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:43:29,743 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-07-26 11:43:29,744 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:43:29,747 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-07-26 11:43:29,748 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:43:29,749 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("f1")
2025-07-26 11:43:29,751 INFO

ValueError: Must provide exactly one of 'toolkit' or 'db'. Received neither.

# Algunos ejemplos de consultas

In [ ]:
response = agente.invoke({"input":"¿Cuántos pilotos que ganaron mas de campeonato mundial hay en la base de datos?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Cuántos campeonatos mundiales tiene Ayrton Senna?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Qué porcentaje de carreras se corrieron bajo la lluvia?"})
print(response['output'])

In [ ]:
response = agente.invoke({"input":"¿Cómo se relaciona la posicion final con los puntos obtenidos?"})
print(response['output'])